In [ ]:
import PIL
from PIL import Image
import numpy as np
from PIL import ImageFilter
from __future__ import print_function
import pandas as pd
import multiprocessing as mp
import time
import matplotlib.pyplot as plt

import h5py
import pandas as pd

h5y_r = h5py.File('intput_path', 'r')
X_data = h5y_r['OM_x'][:]/255
print(X_data.shape)
h5y_r.close()

h5y_r = h5py.File('intput_path', 'r')
Y_data = h5y_r['SEM_y'][:]/255
print(Y_data.shape)
h5y_r.close()

#%%
#index separation
tot_ix =range(len(X_data))
test_ix=np.random.choice(tot_ix, round(len(X_data)*0.2))

mat1 = test_ix
dataframe1 = pd.DataFrame(data=mat1.astype(int))
dataframe1.to_csv('intput_path', sep=',', header=False, index=False)

train_ix = list(set(tot_ix) - set(test_ix))
#%%
#data separate

test_ix = pd.read_csv('intput_path',header=None)
test_ix= test_ix[0].tolist()
tot_ix =range(len(X_data))
train_ix = list(set(tot_ix) - set(test_ix))

train_X=X_data[train_ix]
train_Y=Y_data[train_ix]
test_X=X_data[test_ix]
test_Y=Y_data[test_ix]

print(len(train_X))
print(len(train_Y))

#%%
runfile('intput_path', wdir='D://cyclegan')

n_epoch = 3000
batch_size = 5

#import tqdm
losses = {"d_X":[], "d_Y":[], "gan_XY":[]}

for i in range(n_epoch) :
    batch1 =  np.random.choice(range(len(train_X)), batch_size)
    batch2 =  np.random.choice(range(len(train_Y)), batch_size)
    X_ = train_X[batch1]
    Y_ = train_Y[batch2]
    
    Label = np.zeros([2*batch_size,15,15,1])
    Label[0:batch_size] = 1
    Label[batch_size:] = 0
    
    G_X_image = G_model.predict(X_)
    Y_ = np.concatenate((Y_, G_X_image))

    d_Y_loss = D_Y_model.train_on_batch(Y_, Label)
    losses["d_Y"].append(d_Y_loss)
    
    F_Y_image = F_model.predict(Y_[0:batch_size])
    X_ = np.concatenate((X_, F_Y_image))

    d_X_loss = D_X_model.train_on_batch(X_, Label)
    losses["d_X"].append(d_X_loss)
        
    X_ = train_X[batch1]
    Y_ = train_Y[batch2]
    
    Label = Label[0:batch_size]
    
    gan_XY_loss = GAN_XY_model.train_on_batch([X_,Y_],[X_, Y_, Label, Label])
    losses["gan_XY"].append(gan_XY_loss)
    
    if i%10 == 0: 
        print(i, d_X_loss, d_Y_loss, gan_XY_loss[4])
    
    if i%100 == 0:
        print(np.mean(D_X_model.predict(X_)), np.mean(D_X_model.predict(F_Y_image)))
        print(np.mean(D_Y_model.predict(Y_)), np.mean(D_Y_model.predict(G_X_image)))
        plt.imshow(np.squeeze(np.uint8(F_model.predict(test_Y)[10]*255)))
        plt.show()

G_predicted = G_model.predict(test_X)*255
F_predicted = F_model.predict(test_Y)*255
F_model.save_weights('save_path'+str(2300)+'.out')
G_model.save_weights('save_path'+str(2300)+'.out')

t1 = np.squeeze(np.uint8(train_Y[1]*255)); t2 = np.squeeze(np.uint8(F_model.predict(train_Y)[1]*255))
t3 = np.squeeze(np.uint8(G_model.predict(F_model.predict(train_Y))[1]*255))
plt.imshow(t1);plt.show();plt.imshow(t2);plt.show();plt.imshow(t3);plt.show()

t1 = np.squeeze(np.uint8(train_X[1]*255)); t2 = np.squeeze(np.uint8(F_model.predict(train_X)[1]*255))
t3 = np.squeeze(np.uint8(G_model.predict(F_model.predict(train_X))[1]*255))
plt.imshow(t1);plt.show();plt.imshow(t2);plt.show();plt.imshow(t3);plt.show()

for i in range(100):
    a128=np.uint8(G_predicted[i])
    a128=np.squeeze(a128)
    test_img=Image.fromarray(a128)
    test_img.save("save_path"+str(i)+".png")

for i in range(100):
    a128=np.uint8(F_predicted[i])
    a128=np.squeeze(a128)
    test_img=Image.fromarray(a128)
    test_img.save("save_path"+str(i)+".png")

for i in range(100):
    a128=np.uint8(test_X[i]*255)
    a128=np.squeeze(a128)
    test_img=Image.fromarray(a128)
    test_img.save("save_path"+str(i)+".png")      
    
for i in range(100):
    a128=np.uint8(test_Y[i]*255)
    a128=np.squeeze(a128)
    test_img=Image.fromarray(a128)
    test_img.save("save_path"+str(i)+".png")

In [ ]:
import keras.backend as K
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Merge, Concatenate ,Merge , Add
from keras.layers import Dropout, Activation, LeakyReLU, PReLU, ELU, ThresholdedReLU
from keras.layers import Conv1D, Conv2D, MaxPooling2D, AveragePooling1D, AveragePooling2D, ZeroPadding2D
from keras.layers import ZeroPadding2D, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers import Input, Dense, TimeDistributed, GRU
from keras.layers.normalization import BatchNormalization
from keras.engine.topology import Container
#from keras_contrib.layers.normalization import InstanceNormalization
from keras.initializers import RandomNormal
from keras.layers import LSTM, Reshape
import keras.callbacks
from keras.optimizers import *
import numpy as np

opt = Adam(lr=1e-4)
dopt = Adam(lr=1e-5)
cycle_loss_weight = 8
flag = 'mix2'

def discriminator_model():
    
    x_input = Input(shape=(256,256,1))
    
    x = ZeroPadding2D(padding=(1, 1))(x_input)
    x = Conv2D(64, (4, 4), strides=(2, 2), padding='valid', kernel_initializer = RandomNormal(0, 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    
    x = Conv2D(2*64, (4, 4), strides=(2, 2), padding='valid', kernel_initializer = RandomNormal(0, 0.02))(x)
    #x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    
    x = Conv2D(4*64, (4, 4), strides=(2, 2), padding='valid', kernel_initializer = RandomNormal(0, 0.02))(x)
    #x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    
    x = Conv2D(6*64, (4, 4), strides=(2, 2), padding='valid', kernel_initializer = RandomNormal(0, 0.02))(x)
    #x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    

    x = Conv2D(1, (4, 4), activation='sigmoid', strides=(1, 1))(x)
    
    #x = Flatten()(x)
    #x = Dense(2,activation='softmax')(x)
    
    final_model = Model(inputs = x_input, outputs = x)
    final_model.compile(optimizer = dopt, loss = 'mse')
    
    return final_model 

def generator_model(flag):
    
    x_input = Input(shape=(256,256,1))
    #CONV block
    x = Conv2D(128, (7, 7), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x_input)
    x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
    x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
    x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
    x = LeakyReLU(alpha=0.2)(x)

    for i in range(6):
        x = residual_block(x)

    if flag == 'deconv':
        x = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
    if flag == 'upsample' :
        x = UpSampling2D()(x)
        x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
        x = UpSampling2D()(x)
        x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
    if flag == 'mix' :
        x = UpSampling2D()(x)
        x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
    if flag == 'mix1' :
        x = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
        x = UpSampling2D()(x)
        x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(x)
        x = Activation('relu')(x)
    if flag == 'mix2' :
        x = UpSampling2D()(x)
        x = Conv2D(256, (2, 2), strides = (1, 1), padding = 'same', kernel_initializer = RandomNormal(0, 0.03))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.03))(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same', kernel_initializer = RandomNormal(0, 0.03))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.03))(x)
        x = Activation('relu')(x)
        x = Conv2D(128, (2, 2), strides = (1, 1), padding = 'same', kernel_initializer = RandomNormal(0, 0.03))(x)
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.03))(x)
        x = Activation('relu')(x)
        x = Conv2D(128, (2, 2), strides = (1, 1), padding = 'same', kernel_initializer = RandomNormal(0, 0.03))(x) 
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.03))(x)
        x = Activation('relu')(x)
        x = Conv2D(64, (2, 2), strides = (1, 1), padding = 'same', kernel_initializer = RandomNormal(0, 0.03))(x) 
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.03))(x)
        x = Activation('relu')(x)
        x = Conv2D(64, (2, 2), strides = (1, 1), padding = 'same', kernel_initializer = RandomNormal(0, 0.03))(x) 
        x = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.03))(x)
        x = Activation('relu')(x)
        
    conv_outputs = Conv2D(1, (7, 7), activation='tanh', strides=(1, 1) ,padding='same')(x)  
    
    final_model = Model(inputs = x_input, outputs = conv_outputs)
    
    return final_model, x_input, conv_outputs

def residual_block(x):
    y = Conv2D(256, (3, 3), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(x)
    y = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(y)
    y = LeakyReLU(alpha=0.2)(y)
    y = Conv2D(256, (3, 3), strides=(1, 1), padding='same', kernel_initializer = RandomNormal(0, 0.02))(y)
    y = BatchNormalization(momentum=0.9, axis=3, epsilon=1e-5, gamma_initializer = RandomNormal(1., 0.02))(y)
    y = Activation('relu')(y)
    return Add()([y, x])

def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

D_X_model = discriminator_model()
D_Y_model = discriminator_model()

G_model, realX, fakeY = generator_model(flag)
F_model, realY, fakeX = generator_model(flag)

make_trainable(D_X_model, False)
make_trainable(D_Y_model, False)

GAN_XY_model = Model(inputs = [realX, realY], 
                     outputs = [  F_model(fakeY), G_model(fakeX), D_X_model(fakeX),  D_Y_model(fakeY)])

GAN_XY_model.compile(optimizer=opt, loss=['mae','mae','mse','mse'], 
                     loss_weights=[cycle_loss_weight, cycle_loss_weight, 1., 1.])

print(len(D_Y_model.trainable_weights), len(D_Y_model._collected_trainable_weights))
print(len(GAN_XY_model.trainable_weights),len(GAN_XY_model._collected_trainable_weights))

GAN_XY_model.summary()


